#### Installation code for the Colab session

In [ ]:
pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 42.9 MB/s eta 0:00:00


In [ ]:
pip install rioxarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.9 MB/s eta 0:00:00


In [ ]:
pip install xarray-spatial

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 KB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for datashape: filename=datashape-0.5.2-py3-none-any.whl size=59439 sha256=d16347615f706ad3fd7dc08e00a1fec68ecdc70675bb58a6717748495bf743a9
  Stored in directory: /root/.cache/pip/wheels/6d/79/c4/c425774559165f472d32e5ef592ff9a71179abb31f05dbc98b
Successfully built datashape


In [ ]:
pip install pystac-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 7.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [ ]:
pip install planetary-computer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install odc-stac

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 KB 13.4 MB/s eta 0:00:00


In [ ]:
pip install odc-algo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 KB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.2/371.2 KB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.3/555.3 KB 38.8 MB/s eta 0:00:00
  Created wheel for dask-image: filename=dask_image-2022.9.0-py2.py3-none-any.whl size=41053 sha256=140d40ac7cd85fdee63939ca35a29d6a947dcf71770813e1e0f5d71aa86b3331
  Stored 

#### Load the necessary libraries

In [ ]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import rasterio.features
import rioxarray as rio
import xrspatial.multispectral as ms

# Import Planetary Computer tools
import pystac_client
import planetary_computer as pc
pc.settings.set_subscription_key('99d0028b4d864147958f0b6b44663e5f')
#import odc
from odc.stac import stac_load
from odc.algo import to_rgba
from tqdm import tqdm

# For finetuning ResNet-18
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import time

#### Get the data from drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
crop_presence_data = pd.read_csv("/content/drive/MyDrive/Crop_Location_Data_20221201.csv")

In [ ]:
crop_presence_data["Latitude"] = crop_presence_data["Latitude and Longitude"].apply(lambda x: float(x[1:-1].split(",")[0]))
crop_presence_data["Longitude"] = crop_presence_data["Latitude and Longitude"].apply(lambda x: float(x[1:-1].split(",")[1]))

In [ ]:
crop_presence_data["Class"] = crop_presence_data["Class of Land"].apply(lambda x: 1 if x == "Rice" else 0)

In [ ]:
crop_presence_data.head()

,Latitude and Longitude,Class of Land,Latitude,Longitude,Class
0,"(10.323727047081501, 105.2516346045924)",Rice,10.323727,105.251635,1
1,"(10.322364360592521, 105.27843410554115)",Rice,10.322364,105.278434,1
2,"(10.321455902933202, 105.25254306225168)",Rice,10.321456,105.252543,1
3,"(10.324181275911162, 105.25118037576274)",Rice,10.324181,105.251180,1
4,"(10.324635504740822, 105.27389181724476)",Rice,10.324636,105.273892,1


#### Get satellite data based on latitude and longitude

In [ ]:
box_size_deg = 0.10
resolution = 20  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for CRS:4326 
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [ ]:
def get_data_latlong(lat, long, time_window):
    min_lon = long-box_size_deg/2
    min_lat = lat-box_size_deg/2
    max_lon = long+box_size_deg/2
    max_lat = lat+box_size_deg/2
    bounds = (min_lon, min_lat, max_lon, max_lat)
    search = stac.search(collections=["sentinel-2-l2a"], bbox=bounds, datetime=time_window)
    items = list(search.get_all_items())
    xx = stac_load(
        items,
        bands=["red", "green", "blue", "nir", "SCL"],
        crs="EPSG:4326", # Latitude-Longitude
        resolution=scale, # Degrees
        chunks={"x": 2048, "y": 2048},
        dtype="uint16",
        patch_url=pc.sign,
        bbox=bounds
    )
    return xx

In [ ]:
lat_arr = crop_presence_data["Latitude"]
long_arr = crop_presence_data["Longitude"]

In [ ]:
xx_lst_1 = [0]*len(crop_presence_data)
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
for inx in tqdm(range(len(crop_presence_data)), desc = "Running"):
    xx = get_data_latlong(lat_arr[inx], long_arr[inx], "2021-05-29/2021-05-30")
    xx_lst_1[inx] = xx

Running: 100%|██████████| 600/600 [02:42<00:00,  3.69it/s]


In [ ]:
xx_lst_2 = [0]*len(crop_presence_data)
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
for inx in tqdm(range(len(crop_presence_data)), desc = "Running"):
    xx = get_data_latlong(lat_arr[inx], long_arr[inx], "2021-07-29/2021-07-30")
    xx_lst_2[inx] = xx

Running: 100%|██████████| 600/600 [02:43<00:00,  3.67it/s]


In [ ]:
xx_lst_3 = [0]*len(crop_presence_data)
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
for inx in tqdm(range(len(crop_presence_data)), desc = "Running"):
    xx = get_data_latlong(lat_arr[inx], long_arr[inx], "2021-08-27/2021-08-28")
    xx_lst_3[inx] = xx

Running: 100%|██████████| 600/600 [02:44<00:00,  3.65it/s]


Need to divide 600 location to 3 batchs in order to make the code works

In [ ]:
xx_lst_np_11 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_1[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_11[inx] = temp.data.compute() # output: xarray.dataArray

Running: 100%|██████████| 200/200 [02:42<00:00,  1.23it/s]


In [ ]:
xx_lst_np_12 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_1[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_12[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [02:55<00:00,  1.14it/s]


In [ ]:
xx_lst_np_13 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_1[400+inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_13[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [02:41<00:00,  1.23it/s]


In [ ]:
xx_lst_np_21 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_2[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_21[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [02:09<00:00,  1.54it/s]


In [ ]:
xx_lst_np_22 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_2[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_22[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [05:30<00:00,  1.65s/it]


In [ ]:
xx_lst_np_23 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_2[400+inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_23[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [05:22<00:00,  1.61s/it]


In [ ]:
xx_lst_np_31 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_3[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_31[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [02:21<00:00,  1.41it/s]


In [ ]:
xx_lst_np_32 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_3[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_32[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [02:18<00:00,  1.45it/s]


In [ ]:
xx_lst_np_33 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_3[400+inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_33[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [02:28<00:00,  1.35it/s]


In [ ]:
# Convert this to this size before using on any model 
for inx in range(200):
    xx_lst_np_11[inx] = np.resize(xx_lst_np_11[inx], (3, 557, 557))
    xx_lst_np_12[inx] = np.resize(xx_lst_np_12[inx], (3, 557, 557))
    xx_lst_np_13[inx] = np.resize(xx_lst_np_13[inx], (3, 557, 557))
    xx_lst_np_21[inx] = np.resize(xx_lst_np_11[inx], (3, 557, 557))
    xx_lst_np_22[inx] = np.resize(xx_lst_np_12[inx], (3, 557, 557))
    xx_lst_np_23[inx] = np.resize(xx_lst_np_13[inx], (3, 557, 557))
    xx_lst_np_31[inx] = np.resize(xx_lst_np_11[inx], (3, 557, 557))
    xx_lst_np_32[inx] = np.resize(xx_lst_np_12[inx], (3, 557, 557))
    xx_lst_np_33[inx] = np.resize(xx_lst_np_13[inx], (3, 557, 557))

#### Training and Testing using Resnet-18

Concat and transform the data to right format

In [ ]:
data = xx_lst_np_11 + xx_lst_np_12 + xx_lst_np_13 + xx_lst_np_21 + xx_lst_np_22 + xx_lst_np_23 + xx_lst_np_31 + xx_lst_np_32 + xx_lst_np_33

In [ ]:
transformations = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
for i in range(1800):
  data[i] = transformations(data[i].reshape(557,557,3).astype('uint8'))

In [ ]:
import random
# get sample index in order to divide into train and test set
num_index = 1800
random_sample_80 = random.sample(range(num_index), int(num_index * 0.8))
train_arr = []
test_arr = []
train_label = []
test_label = []
for i in range(num_index):
  if i in random_sample_80:
    train_arr.append(data[i])
    train_label.append(crop_presence_data["Class"].iloc[i % 600])
  else:
    test_arr.append(data[i])
    test_label.append(crop_presence_data["Class"].iloc[i % 600])

Create the train and testing data + label, then make it into a training and testing dataset, then convert it to dataloader as it will be divided into many batches.

In [ ]:
train_arr = torch.stack(train_arr)
test_arr = torch.stack(test_arr)

In [ ]:
train_label = torch.tensor(np.array(train_label))
test_label = torch.tensor(np.array(test_label))

In [ ]:
train_arr = torch.FloatTensor(train_arr)
train_label = torch.LongTensor(train_label)
test_arr = torch.FloatTensor(test_arr)
test_label = torch.LongTensor(test_label)

In [ ]:
train_dataset = TensorDataset(train_arr, train_label)
test_dataset = TensorDataset(test_arr, test_label)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=8)

Load the model and train the model

In [ ]:
model = models.resnet18(pretrained=True)   #load resnet18 model
num_features = model.fc.in_features     #extract fc layers features
model.fc = nn.Linear(num_features, 2) #(num_of_class == 2)
model = model.to(device) 
criterion = nn.CrossEntropyLoss()  #(set loss function)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 60   #(set no of epochs)
start_time = time.time() #(for showing time)
for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device) 
        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))
    
    """ Testing Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(test_dataset)
        epoch_acc = running_corrects / len(test_dataset) * 100.
        print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))

Epoch 0 running
[Train #0] Loss: 0.4973 Acc: 74.3750% Time: 13.8975s
[Test #0] Loss: 0.3174 Acc: 83.0556% Time: 15.7656s
Epoch 1 running
[Train #1] Loss: 0.3329 Acc: 81.8056% Time: 22.0835s
[Test #1] Loss: 0.3126 Acc: 82.2222% Time: 24.0969s
Epoch 2 running
[Train #2] Loss: 0.3049 Acc: 81.3194% Time: 29.8965s
[Test #2] Loss: 0.2903 Acc: 83.6111% Time: 31.7732s
Epoch 3 running
[Train #3] Loss: 0.2851 Acc: 82.4306% Time: 38.1571s
[Test #3] Loss: 0.2873 Acc: 78.6111% Time: 40.4766s
Epoch 4 running
[Train #4] Loss: 0.2750 Acc: 82.7778% Time: 46.3208s
[Test #4] Loss: 0.2902 Acc: 78.0556% Time: 50.2598s
Epoch 5 running
[Train #5] Loss: 0.2760 Acc: 82.8472% Time: 56.5657s
[Test #5] Loss: 0.3048 Acc: 78.3333% Time: 58.3958s
Epoch 6 running
[Train #6] Loss: 0.2769 Acc: 81.3194% Time: 64.7663s
[Test #6] Loss: 0.2994 Acc: 83.6111% Time: 66.8962s
Epoch 7 running
[Train #7] Loss: 0.2653 Acc: 82.1528% Time: 72.7892s
[Test #7] Loss: 0.2966 Acc: 78.3333% Time: 74.6951s
Epoch 8 running
[Train #8] Loss:

Save the model as a pth file to use it later

In [ ]:
save_path = '/content/drive/MyDrive/resnet18-3daydata-eychallenge.pth'
torch.save(model.state_dict(), save_path)

Testing process of the model

In [ ]:
model = models.resnet18(pretrained=True)   #load resnet18 model
num_features = model.fc.in_features #extract fc layers features
model.fc = nn.Linear(num_features, 2)#(num_of_class == 2)
model.load_state_dict(torch.load('/content/drive/MyDrive/resnet18-3daydata-eychallenge.pth'))
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
model.eval()
start_time = time.time()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    for i, (inputs, labels) in enumerate(test_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects / len(test_dataset) * 100.
    print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.
          format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

[Test #59] Loss: 0.3880 Acc: 76.1111% Time: 2.0615s


#### Training and Testing using Decision Tree and Logistic Regression

In [ ]:
# Now convert to 1D array to use
for inx in range(200):
    xx_lst_np_11[inx] = xx_lst_np_11[inx].reshape(3*557*557)
    xx_lst_np_12[inx] = xx_lst_np_12[inx].reshape(3*557*557)
    xx_lst_np_13[inx] = xx_lst_np_13[inx].reshape(3*557*557)

In [ ]:
x = np.concatenate([np.array(xx_lst_np_11), np.array(xx_lst_np_12), np.array(xx_lst_np_13)], axis = 0)
# np.array(xx_lst_np_21), np.array(xx_lst_np_22), np.array(xx_lst_np_23)
# we now have 2D array

In [ ]:
from sklearn.preprocessing import MinMaxScaler

Scaler = MinMaxScaler()
x = Scaler.fit_transform(x)

In [ ]:
y = np.concatenate([np.array(crop_presence_data["Class"])])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 101)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression()

In [ ]:
LR.fit(x_train, y_train)

In [ ]:
pred_LR = LR.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, pred_LR))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

In [ ]:
clf.fit(x_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred_clf = clf.predict(x_test)

In [ ]:
print(classification_report(y_test, pred_clf))

              precision    recall  f1-score   support

           0       0.64      0.78      0.70       166
           1       0.77      0.62      0.69       194

    accuracy                           0.69       360
   macro avg       0.70      0.70      0.69       360
weighted avg       0.71      0.69      0.69       360



#### Make prediction on challenge's data

Load the challenge's test data

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/challenge_1_submission_template_correct_columns_fixed.csv")

In [ ]:
test_data.head()

,id,target
0,"(10.18019073690894, 105.32022315786804)",NaN
1,"(10.561107033461816, 105.12772097986661)",NaN
2,"(10.623790611954897, 105.13771401411867)",NaN
3,"(10.583364246115156, 105.23946127195805)",NaN
4,"(10.20744446668854, 105.26844107128906)",NaN


In [ ]:
test_data["Latitude"] = test_data["id"].apply(lambda x: float(x[1:-1].split(",")[0]))
test_data["Longitude"] = test_data["id"].apply(lambda x: float(x[1:-1].split(",")[1]))

In [ ]:
test_data.head()

,id,target,Latitude,Longitude
0,"(10.18019073690894, 105.32022315786804)",NaN,10.180191,105.320223
1,"(10.561107033461816, 105.12772097986661)",NaN,10.561107,105.127721
2,"(10.623790611954897, 105.13771401411867)",NaN,10.623791,105.137714
3,"(10.583364246115156, 105.23946127195805)",NaN,10.583364,105.239461
4,"(10.20744446668854, 105.26844107128906)",NaN,10.207444,105.268441


In [ ]:
lat_arr_test = test_data["Latitude"]
long_arr_test = test_data["Longitude"]

Get the satellite data of new location

In [ ]:
test_lst_1 = [0]*250
for inx in tqdm(range(250), desc="Running"):
    xx = get_data_latlong(lat_arr_test[inx], long_arr_test[inx], "2022-05-29/2022-05-30")
    test_lst_1[inx] = xx

Running: 100%|██████████| 250/250 [01:13<00:00,  3.39it/s]


In [ ]:
test_lst_2 = [0]*250
for inx in tqdm(range(250), desc="Running"):
    xx = get_data_latlong(lat_arr_test[inx], long_arr_test[inx], "2022-07-29/2022-07-30")
    test_lst_2[inx] = xx

Running: 100%|██████████| 250/250 [01:10<00:00,  3.52it/s]


In [ ]:
test_lst_3 = [0]*250
for inx in tqdm(range(250), desc="Running"):
    xx = get_data_latlong(lat_arr_test[inx], long_arr_test[inx], "2022-08-23/2022-08-24")
    test_lst_3[inx] = xx

Running: 100%|██████████| 250/250 [01:09<00:00,  3.60it/s]


In [ ]:
test_lst_np_11 = [0] * 125
for inx in tqdm(range(125), desc="Running"):
    temp = test_lst_1[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    test_lst_np_11[inx] = temp.data.compute()

Running: 100%|██████████| 125/125 [02:14<00:00,  1.08s/it]


In [ ]:
test_lst_np_12 = [0] * 125
for inx in tqdm(range(125), desc="Running"):
    temp = test_lst_1[125+inx].isel(time=0)[["red", "green", "blue"]].to_array()
    test_lst_np_12[inx] = temp.data.compute()

Running:  30%|███       | 38/125 [03:42<08:29,  5.86s/it]


KeyboardInterrupt: ignored

In [ ]:
test_lst_np_21 = [0] * 125
for inx in tqdm(range(125), desc="Running"):
    temp = test_lst_2[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    test_lst_np_21[inx] = temp.data.compute()

Running: 100%|██████████| 125/125 [02:11<00:00,  1.05s/it]


In [ ]:
test_lst_np_22 = [0] * 125
for inx in tqdm(range(125), desc="Running"):
    temp = test_lst_2[125+inx].isel(time=0)[["red", "green", "blue"]].to_array()
    test_lst_np_22[inx] = temp.data.compute()

Running: 100%|██████████| 125/125 [02:48<00:00,  1.35s/it]


In [ ]:
test_lst_np_31 = [0] * 125
for inx in tqdm(range(125), desc="Running"):
    temp = test_lst_3[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    test_lst_np_31[inx] = temp.data.compute()

Running: 100%|██████████| 125/125 [02:58<00:00,  1.43s/it]


In [ ]:
test_lst_np_32 = [0] * 125
for inx in tqdm(range(125), desc="Running"):
    temp = test_lst_3[125+inx].isel(time=0)[["red", "green", "blue"]].to_array()
    test_lst_np_32[inx] = temp.data.compute()

Running: 100%|██████████| 125/125 [02:55<00:00,  1.40s/it]


In [ ]:
for inx in range(125):
    test_lst_np_11[inx] = np.resize(test_lst_np_11[inx], (3, 557, 557))
    test_lst_np_12[inx] = np.resize(test_lst_np_12[inx], (3, 557, 557))
    test_lst_np_21[inx] = np.resize(test_lst_np_21[inx], (3, 557, 557))
    test_lst_np_22[inx] = np.resize(test_lst_np_22[inx], (3, 557, 557))
    test_lst_np_31[inx] = np.resize(test_lst_np_31[inx], (3, 557, 557))
    test_lst_np_32[inx] = np.resize(test_lst_np_32[inx], (3, 557, 557))

Use resnet-18 to predict

In [ ]:
my_test_data_1 = test_lst_np_11 + test_lst_np_12
my_test_data_2 = test_lst_np_21 + test_lst_np_22
my_test_data_3 = test_lst_np_31 + test_lst_np_32

In [ ]:
transformations = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
for i in range(250):
  my_test_data_1[i] = transformations(my_test_data_1[i].reshape(557,557,3).astype('uint8'))
  my_test_data_2[i] = transformations(my_test_data_2[i].reshape(557,557,3).astype('uint8'))
  my_test_data_3[i] = transformations(my_test_data_3[i].reshape(557,557,3).astype('uint8'))

In [ ]:
my_test_arr_1 = torch.stack(my_test_data_1)
my_test_arr_1 = torch.FloatTensor(my_test_arr_1)
my_test_arr_2 = torch.stack(my_test_data_2)
my_test_arr_2 = torch.FloatTensor(my_test_arr_2)
my_test_arr_3 = torch.stack(my_test_data_3)
my_test_arr_3 = torch.FloatTensor(my_test_arr_3)

In [ ]:
my_dummy_label = [0] * 250
my_dummy_label = torch.LongTensor(np.array(my_dummy_label))

In [ ]:
my_test_dataset_1 = TensorDataset(my_test_arr_1, my_dummy_label)
my_test_dataset_2 = TensorDataset(my_test_arr_2, my_dummy_label)
my_test_dataset_3 = TensorDataset(my_test_arr_3, my_dummy_label)

In [ ]:
my_test_dataloader_1 = torch.utils.data.DataLoader(my_test_dataset_1, batch_size=32, shuffle=False, num_workers=8)
my_test_dataloader_2 = torch.utils.data.DataLoader(my_test_dataset_2, batch_size=32, shuffle=False, num_workers=8)
my_test_dataloader_3 = torch.utils.data.DataLoader(my_test_dataset_3, batch_size=32, shuffle=False, num_workers=8)

In [ ]:
# Loop over the DataLoader and pass the data through the model
def predict(aDataloader):
  predictions = []
  with torch.no_grad():
      for i, (inputs, labels) in enumerate(aDataloader):
          inputs = inputs.to(device)
          labels = labels.to(device)
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          predictions += preds.tolist()
  return predictions

In [ ]:
pred1 = predict(my_test_dataloader_1)

In [ ]:
pred2 = predict(my_test_dataloader_2)

In [ ]:
pred3 = predict(my_test_dataloader_3)

In [ ]:
df = pd.DataFrame({"target1": pred1, "target2" : pred2, "target3": pred3})

In [ ]:
df["target"] = (df["target1"] + df["target2"] + df["target3"]) / 3
df["target"] = df["target"].apply(lambda x: 1 if x > 0.5 else 0)

In [ ]:
df.head()

,target1,target2,target3,target
0,1,0,1,1
1,0,0,0,0
2,0,0,0,0
3,1,0,0,0
4,1,0,1,1


In [ ]:
test_data["target"] = df["target"]

In [ ]:
test_data.head()

,id,target,Latitude,Longitude
0,"(10.18019073690894, 105.32022315786804)",1,10.180191,105.320223
1,"(10.561107033461816, 105.12772097986661)",0,10.561107,105.127721
2,"(10.623790611954897, 105.13771401411867)",0,10.623791,105.137714
3,"(10.583364246115156, 105.23946127195805)",0,10.583364,105.239461
4,"(10.20744446668854, 105.26844107128906)",1,10.207444,105.268441


In [ ]:
export_RN = test_data[["id", "target"]]

In [ ]:
export_RN["target"] = export_RN["target"].apply(lambda x: "Rice" if x == 1 else "Non Rice")

In [ ]:
export_RN.to_csv("/content/drive/MyDrive/resnet_3day.csv", index = False)

Use DT and LR for predict

In [ ]:
for inx in range(125):
    test_lst_np_1[inx] = np.resize(test_lst_np_1[inx], (3, 557, 557)).reshape(3*557*557)
    test_lst_np_2[inx] = np.resize(test_lst_np_2[inx], (3, 557, 557)).reshape(3*557*557)

In [ ]:
my_test = np.concatenate([np.array(test_lst_np_1), np.array(test_lst_np_2)], axis = 0)

In [ ]:
pred_test = clf.predict(my_test)

In [ ]:
len(pred_test)

In [ ]:
test_data["target"] = pd.DataFrame({"target": pred_test})

In [ ]:
test_data.head()

In [ ]:
export = test_data[["id", "target"]]

In [ ]:
export["target"] = export["target"].apply(lambda x: "Rice" if x == 1 else "Non Rice")

In [ ]:
export.to_csv(".../Test_result_colab.csv", index = False)